In [1]:
import os
import os.path as osp
import networkx as nx
import json
import matplotlib.pyplot as plt
import csv
import pandas as pd
import subprocess
# build graph for game
from utils.map_utils_old import get_game_info,get_edge
from utils.clean_utils import compute_hash_for_path,compute_hash_for_pair

In [2]:
# note book input
map_dir='/share/data/mei-work/kangrui/github/mango/data-intermediate/'
new_map_dir='/share/data/mei-work/kangrui/github/mango/data/'

In [3]:

def replace_str_in_dict(obj, replace_dict):
    if isinstance(obj, dict):
        return {k: replace_str_in_dict(v, replace_dict) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [replace_str_in_dict(i, replace_dict) for i in obj]
    elif isinstance(obj, str):
        if obj in replace_dict.keys():
            
            obj = replace_dict[obj]
        return obj
    else:
        return obj
    

In [4]:
for game_name in os.listdir(map_dir):
    
    try:
        G,all2all,all_pairs,anno2code,code2anno=get_game_info(map_dir,game_name)
    except Exception as e:
        print(e)
        continue
    
    edges=[]
    for edge in G.edges(data=True):
        edges.append({
            "prev_node":edge[0],
            "node":edge[1],
            "action":edge[2]['action'],
            "seen_in_forward": True if edge[2]['desc']=='forward' else False,
            "step_min_cutoff":edge[2]['step_min_cutoff'],
        })
        
    edges=replace_str_in_dict(edges, code2anno)
    all2all_new=replace_str_in_dict(all2all, code2anno)
    all_pairs_new=replace_str_in_dict(all_pairs, code2anno)
    locations=[k for k in anno2code.keys()]
    
    all2all_new_new=[]
    for path in all2all_new:
        hashcode=compute_hash_for_path(path["path_details"])
        path['id']=hashcode
        all2all_new_new.append(path)
    
    all_pairs_new_new=[]
    for pair in all_pairs_new:
        hashcode=compute_hash_for_pair(pair)
        pair['id']=hashcode
        if pair["num_paths"]>=1: # valid pair
            all_pairs_new_new.append(pair)
            
    all2all_path=osp.join(new_map_dir,game_name,f"{game_name}.all2all.json")
    all_pairs_path=osp.join(new_map_dir,game_name,f"{game_name}.all_pairs.json")
    location_path=osp.join(new_map_dir,game_name,f"{game_name}.locations.json")
    edges_path=osp.join(new_map_dir,game_name,f"{game_name}.edges.json")
    
    walkthrough_path_old=osp.join(map_dir,game_name,f"{game_name}.walkthrough")
    step_path_old=osp.join(map_dir,game_name,f"{game_name}.moves")
    step_path_new=osp.join(new_map_dir,game_name,f"{game_name}.actions.json")
    
    with open(step_path_old, 'r') as file:
        actions = [line.strip() for line in file]
    
    if not osp.exists(osp.join(new_map_dir,game_name)):
        os.makedirs(osp.join(new_map_dir,game_name))
    
    with open(all2all_path, 'w') as json_file:
        json.dump(all2all_new_new, json_file,indent=4)
        
    with open(all_pairs_path, 'w') as json_file:
        json.dump(all_pairs_new_new, json_file,indent=4)
        
    with open(edges_path, 'w') as json_file:
        json.dump(edges, json_file,indent=4)
        
    with open(location_path, 'w') as json_file:
        json.dump(locations, json_file,indent=4)
        
    with open(step_path_new, 'w') as json_file:
        json.dump(actions, json_file,indent=4)
    
    
    
    subprocess.run(["cp",walkthrough_path_old, osp.join(new_map_dir,game_name)], check=True)
#     subprocess.run(["rm",osp.join(new_map_dir,game_name,f"{game_name}.moves")], check=True)

[Errno 2] No such file or directory: '/share/data/mei-work/kangrui/github/mango/data-intermediate/theatre/theatre.all2all.json'
[Errno 2] No such file or directory: '/share/data/mei-work/kangrui/github/mango/data-intermediate/weapon/weapon.all2all.json'
[Errno 2] No such file or directory: '/share/data/mei-work/kangrui/github/mango/data-intermediate/acorncourt/acorncourt.all2all.json'
